In [26]:
import torch
from torchvision import models, transforms
from sklearn import svm
import torchinfo

from PIL import Image

# 1. Load the pre-trained EfficientNetB0 model
model = models.efficientnet_b0(pretrained=True)

model.eval()

# 3. Define a transform for image preprocessing (adjust as needed)
transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to EfficientNetB0 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
])

# 4. Function to extract features from an image
def extract_features(image_path):
    image = transform(Image.open(image_path)).unsqueeze(0)  # Add a batch dimension
    with torch.no_grad():
        features = model.features(image)  # Extract features from convolutional layers
        pooled_features = torch.nn.functional.adaptive_avg_pool2d(features, output_size=(1, 1)).squeeze()
        return pooled_features

# 5. Extract features for your dataset




/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
from tqdm import tqdm
import numpy as np
x_train = []
y_train = []
labels = {'Bacterial Spot':0,'Early Blight':1,'Healthy':2,'Late Blight':3,'Septoria Leaf Spot':4,'Yellow Leaf Curl Virus':5}
for i in labels:
    folderPath = os.path.join('/kaggle/input/plant-village-dataset-updated/Tomato/Train',i)
    for j in tqdm(os.listdir(folderPath)):
        if j.lower().endswith((".jpg", ".png", ".jpeg")):
            features=extract_features(os.path.join(folderPath,j))
            x_train.append(features)
            y_train.append(labels[i])
        
for i in labels:
    folderPath = os.path.join('/kaggle/input/plant-village-dataset-updated/Tomato/Test',i)
    for j in tqdm(os.listdir(folderPath)):
        if j.lower().endswith((".jpg", ".png", ".jpeg")):
            features=extract_features(os.path.join(folderPath,j))

            x_train.append(features)
            y_train.append(labels[i])
        
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape,y_train.shape)

x_train,x_test,y_train,y_test=train_test_split(x_train, y_train, test_size=0.2, random_state=42)

100%|██████████| 49/49 [00:02<00:00, 23.16it/s]


(11385, 1280) (11385,)


In [22]:
clf = svm.SVC()
clf.fit(x_train,y_train)
joblib.dump(clf, 'svm_tomato_pytorch.pkl')
y_pred=clf.predict(x_test)
accuracy_deep = accuracy_score(y_test,y_pred)
C = confusion_matrix(y_test,y_pred)

print("Accuracy:", accuracy_deep)
print("Confusion Matrix:")
print(C)

# 7. Use the trained SVM to classify new images



Accuracy: 0.9819938515590689
Confusion Matrix:
[[361   1   0   2   0   0]
 [  4 384   1   2   7   2]
 [  0   1 385   0   0   0]
 [  0   5   1 367   2   0]
 [  3   5   0   0 356   1]
 [  2   1   1   0   0 383]]


In [24]:
with open('/kaggle/working/svm_tomato_pytorch.pkl', 'rb') as f:
    svm_classifier = joblib.load(f, mmap_mode=None)
y_pred2=svm_classifier.predict(x_test)
accuracy_deep = accuracy_score(y_test,y_pred2)
C = confusion_matrix(y_test,y_pred2)

print("Accuracy:", accuracy_deep)
print("Confusion Matrix:")
print(C)



Accuracy: 0.9819938515590689
Confusion Matrix:
[[361   1   0   2   0   0]
 [  4 384   1   2   7   2]
 [  0   1 385   0   0   0]
 [  0   5   1 367   2   0]
 [  3   5   0   0 356   1]
 [  2   1   1   0   0 383]]
